In [38]:
import torch
import torch.nn             as nn
from torch.utils.data       import DataLoader, random_split
from torchvision            import transforms
from torchvision.datasets   import MNIST

import lightning                    as     L
from   lightning.pytorch.callbacks  import EarlyStopping
from   lightning.pytorch.callbacks  import TQDMProgressBar, ModelCheckpoint
from   lightning.pytorch.loggers    import TensorBoardLogger, CSVLogger

In [53]:

class ConvNet(L.LightningModule):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1),             # 28x28 -> 32x26x26
            nn.ReLU(),
            nn.MaxPool2d(2),                    # 32x26x26 -> 32x13x13
            nn.Conv2d(32, 64, 3, 1),            # 32x13x13 -> 64x11x11
            nn.ReLU(),
            nn.MaxPool2d(2),                    # 64x11x11 -> 64x5x5
            nn.Conv2d(64, 128, 3, 1),           # 64x5x5 -> 128x3x3
            nn.ReLU(),
            nn.Conv2d(128, 256, 3, 1),          # 128x3x3 -> 256x1x1
            nn.ReLU(),
            nn.Flatten(),                       # 256x1x1 -> 256
            nn.Linear(256, 128),                # 256 -> 128
            nn.ReLU(),
            nn.Linear(128, 10),                 # 128 -> 10
        )
        self.criteria = nn.CrossEntropyLoss()
        mnist_dataset = MNIST(root='.', train=True, download=True, transform=transforms.ToTensor())
        self.train_dataset, self.val_dataset = random_split(mnist_dataset, [55000, 5000])
        self.test_dataset = MNIST(root='.', train=False, download=True, transform=transforms.ToTensor())

    def forward(self, x):
        return self.model(x)

    def _common_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        return y_hat, self.criteria(y_hat, y)

    def training_step(self, batch, batch_idx):
        y_hat, loss = self._common_step(batch, batch_idx)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)
    
    def validation_step(self, batch, batch_idx):
        y_hat, loss = self._common_step(batch, batch_idx)

        self.log('val_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        y_hat, loss = self._common_step(batch, batch_idx)
        self.log('test_loss', loss)
        return loss
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=32, num_workers=10, shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset,   batch_size=32, num_workers=10, shuffle=False, persistent_workers=True)

    def test_dataloader(self):
        return DataLoader(self.test_dataset,  batch_size=32, num_workers=10, shuffle=False, persistent_workers=True)

In [50]:

# model = ConvNet()
# trainer = L.Trainer(max_epochs=10)
# trainer.fit(model)

In [51]:
train_dataset, val_dataset   = random_split(MNIST(root='.', train=True, transform=transforms.ToTensor()), [55000, 5000])
test_dataset                = MNIST(root='.', train=False, transform=transforms.ToTensor())

train_loader                = DataLoader(train_dataset, batch_size=64, shuffle=True,  num_workers=4, persistent_workers=True)
val_loader                  = DataLoader(val_dataset,   batch_size=64, shuffle=False, num_workers=4, persistent_workers=True)
test_loader                 = DataLoader(test_dataset,  batch_size=64, shuffle=False, num_workers=4, persistent_workers=True)
print(f"Train: {len(train_loader)} batches, Val: {len(val_loader)} batches, Test: {len(test_loader)} batches")

Train: 860 batches, Val: 79 batches, Test: 157 batches


In [54]:
model               = ConvNet()
trainer             = L.Trainer(
                            max_epochs=100, enable_progress_bar=True, devices=1,
                            callbacks=[
                                EarlyStopping(monitor="val_loss", patience=5), 
                                TQDMProgressBar(refresh_rate=10),
                                ModelCheckpoint(filename='{epoch}_{step}_{val_loss:.4f}_val_acc={val_acc:.4f}', save_top_k=3, monitor='val_loss')
                                ],
                            logger=[
                                CSVLogger(        "logs/", name='csv/'),
                                TensorBoardLogger("logs/", name='tensorboard/'),
                                ]
                            )
trainer.fit(model)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name     | Type             | Params
----------------------------------------------
0 | model    | Sequential       | 422 K 
1 | criteria | CrossEntropyLoss | 0     
----------------------------------------------
422 K     Trainable params
0         Non-trainable params
422 K     Total params
1.688     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [41]:
model               = ConvNet()

trainer             = L.Trainer(
                            max_epochs=100, accelerator='mps', devices=1,
                            callbacks=[
                                EarlyStopping(monitor="val_acc", mode="max", patience=20), 
                                TQDMProgressBar(refresh_rate=10),
                                ModelCheckpoint(
                                                # dirpath='logs/checkpoints/',

                                                filename='{epoch}_{step}_{val_loss:.4f}_val_acc={val_acc:.4f}', 
                                                monitor='val_acc', save_top_k=3, mode='max', save_on_train_epoch_end=False), 
                                ],
                            logger=[
                                    CSVLogger(        "logs/", name='csv/'),
                                    TensorBoardLogger("logs/", name='tensorboard/'),
                                    ]

                            
                            )
# trainer = pl.Trainer(max_epochs=2, accelerator='m
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name     | Type             | Params
----------------------------------------------
0 | model    | Sequential       | 422 K 
1 | criteria | CrossEntropyLoss | 0     
----------------------------------------------
422 K     Trainable params
0         Non-trainable params
422 K     Total params
1.688     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

MisconfigurationException: `ModelCheckpoint(monitor='val_acc')` could not find the monitored key in the returned metrics: ['train_loss', 'train_loss_step', 'val_loss', 'epoch', 'step']. HINT: Did you call `log('val_acc', value)` in the `LightningModule`?

In [ ]:
L.

In [26]:
csvlogger.log_dir

'logs/csv/version_3'

In [5]:
trainer.test(dataloaders=test_loader)

/Users/abianche/miniconda3/envs/lightning/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at lightning_logs/convnet/version_4/checkpoints/epoch=31_step=27520_val_loss=0.0480_val_acc=val_acc=0.9922.ckpt
Loaded model weights from the checkpoint at lightning_logs/convnet/version_4/checkpoints/epoch=31_step=27520_val_loss=0.0480_val_acc=val_acc=0.9922.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9937000274658203
        test_loss          0.043489180505275726
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.043489180505275726, 'test_acc': 0.9937000274658203}]

In [6]:
# load best model
# best_model = ConvNet.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
print(trainer.checkpoint_callback.best_model_path)

lightning_logs/convnet/version_4/checkpoints/epoch=31_step=27520_val_loss=0.0480_val_acc=val_acc=0.9922.ckpt


In [7]:

for i in trainer.callback_metrics.keys():
    print(i, trainer.callback_metrics[i].cpu().numpy())

test_loss 0.04348918
test_acc 0.9937


In [8]:
print(trainer.callback_metrics['train_loss'])
# print(trainer.callback_metrics['test_loss'])
print(trainer.callback_metrics['val_loss'])

KeyError: 'train_loss'